In [1]:
import os
import json
import pandas as pd
from datetime import datetime


# Read Files

In [2]:
folder_path = r"C:\Users\bburish\Desktop\Python\BrightData\deloitte_data\2022_detailed"
xls_customers = pd.read_excel(r'C:\Users\bburish\Desktop\Python\BrightData\deloitte_data\Customers_map.xlsx')
priority_customers = pd.read_excel(r'C:\Users\bburish\Desktop\Python\BrightData\deloitte_data\הנהלת חשבונות\Copy of דוח לפי לקוחות 2022.xlsx')
data = []

## Customers without JSON FILES
### and Count the number of Reading Json's

In [3]:
## Customers withous JSON FILES
Read_Jsons = 0
Unread_Jsons = 0
for customer_dir in os.scandir(folder_path):
    if not customer_dir.is_dir():
        continue
    
    for customer_id_dir in os.scandir(customer_dir.path):
        if not customer_id_dir.is_dir():
            continue
        
        json_file = os.path.join(customer_id_dir.path, 'auto_mly_bill_trans.json')
        if not os.path.isfile(json_file):
            print(f"{customer_id_dir.path}")
            Unread_Jsons = Unread_Jsons + 1
        else :
            Read_Jsons = Read_Jsons+1
            

print('Unread_Jsons :' ,Unread_Jsons)             
print('Read_Jsons :' ,Read_Jsons)   
          

       



C:\Users\bburish\Desktop\Python\BrightData\deloitte_data\2022_detailed\1950470\hl_980e51e6
C:\Users\bburish\Desktop\Python\BrightData\deloitte_data\2022_detailed\1985981\hl_14481b7e
C:\Users\bburish\Desktop\Python\BrightData\deloitte_data\2022_detailed\1987770\hl_02dd2f04
Unread_Jsons : 3
Read_Jsons : 95


### check :
##### It's no 'auto_mly_bill_trans.json' files in :
###### C:\Users\bburish\Desktop\Python\BrightData\deloitte_data\2022_detailed\1950470\hl_980e51e6
###### C:\Users\bburish\Desktop\Python\BrightData\deloitte_data\2022_detailed\1985981\hl_14481b7e
###### C:\Users\bburish\Desktop\Python\BrightData\deloitte_data\2022_detailed\1987770\hl_02dd2f04

## Read The json_file to a datframe

In [4]:

for customer_dir in os.scandir(folder_path):
    if not customer_dir.is_dir():
        continue
    
    for customer_id_dir in os.scandir(customer_dir.path):
        if not customer_id_dir.is_dir():
            continue
        
        json_file = os.path.join(customer_id_dir.path, 'auto_mly_bill_trans.json')
        if not os.path.isfile(json_file):
            continue
        
        with open(json_file) as f:
            json_data = json.load(f)
            
              

        for i in json_data:
            try:
                period_from = datetime.strptime(i['period']['from'][:19], '%Y-%m-%dT%H:%M:%S').strftime('%d - %m - %Y')
                period_to = datetime.strptime(i['period']['to'][:19], '%Y-%m-%dT%H:%M:%S').strftime('%d - %m - %Y')
                month = datetime.strptime(period_from, '%d - %m - %Y').strftime('%m')
                year = datetime.strptime(period_from, '%d - %m - %Y').strftime('%Y')
                if year == '2022':
                    try:
                        subtotal = i['calc_cost']['subtotal']
                    except KeyError:
                        subtotal = None
                    customer_id = i.get('customer_id', None)
                    invoice_id = i.get('priority', {}).get('ivnum', None)
                    priority_id = i.get('priority', {}).get('id', None)
                    data.append({
                        'customer_id': customer_id,
                        'Customer Number': customer_dir.name,
                        'invoice_id': invoice_id,
                        'priority_id': priority_id,
                        'month': month,
                        'period_from': period_from,
                        'period_to': period_to,
                        'subtotal_JSON': subtotal
                    })
            except KeyError:
                pass

df = pd.DataFrame(data)


#### Merge the File with Excel File of customers Names (Customers_map) and Priority_customers Excel with Sum חובה and זכות to a new column called Subtotal_Priority
#### and adding new column of differiance between Subtotal_Priority and subtotal_JSON

In [5]:
merged_df = df.merge(xls_customers[['Match 1 - customer_id', 'Nice Name']], left_on='customer_id', right_on='Match 1 - customer_id', how='left')


merged_df = merged_df[['customer_id', 'Nice Name','Customer Number', 'invoice_id', 'priority_id', 'month', 'period_from', 'period_to', 'subtotal_JSON']]

merged_df = pd.merge(merged_df, priority_customers[['פרטים','זכות בדולרים']], left_on='priority_id', right_on='פרטים', how='left')

merge_df = merged_df[['customer_id', 'Nice Name','Customer Number', 'invoice_id', 'priority_id', 'month', 'period_from', 'period_to', 'subtotal_JSON','זכות בדולרים']]

merged_df['Subtotal_Priority'] = merged_df['זכות בדולרים'] 
merged_df.drop(['זכות בדולרים','פרטים'], axis=1, inplace=True)

merged_df['Diff_JSON_Priority'] = abs(merged_df['Subtotal_Priority'] - merged_df['subtotal_JSON'])


In [6]:
merged_df

,customer_id,Nice Name,Customer Number,invoice_id,priority_id,month,period_from,period_to,subtotal_JSON,Subtotal_Priority,Diff_JSON_Priority
0,hl_f68ed861,Summit Racing,1020471,IN224000419,lum223687058618,01,01 - 01 - 2022,01 - 02 - 2022,649.56,649.56,0.00
1,hl_f68ed861,Summit Racing,1020471,IN224000833,lum226106891921,02,01 - 02 - 2022,01 - 03 - 2022,1243.98,1243.98,0.00
2,hl_f68ed861,Summit Racing,1020471,IN224001247,lum228787752664,03,01 - 03 - 2022,01 - 04 - 2022,1371.87,1371.88,0.01
3,hl_f68ed861,Summit Racing,1020471,IN224001654,lum231377954443,04,01 - 04 - 2022,01 - 05 - 2022,1460.47,1460.49,0.02
4,hl_f68ed861,Summit Racing,1020471,IN224002046,lum234043185425,05,01 - 05 - 2022,01 - 06 - 2022,1163.69,1163.69,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1031,hl_b9bf54ba,smartcommerceco,250514,EI228080513,lum241997897804,08,01 - 08 - 2022,01 - 09 - 2022,16333.82,16333.82,0.00
1032,hl_b9bf54ba,smartcommerceco,250514,EI228091296,lum244588803300,09,01 - 09 - 2022,01 - 10 - 2022,20117.77,20117.77,0.00
1033,hl_b9bf54ba,smartcommerceco,250514,EI228101763,lum247267025374,10,01 - 10 - 2022,01 - 11 - 2022,16113.74,16113.75,0.01
1034,hl_b9bf54ba,smartcommerceco,250514,EI228112280,lum249859147138,11,01 - 11 - 2022,01 - 12 - 2022,11830.61,11830.62,0.01


## Checking null values 

In [7]:
merged_df.isna().sum()

customer_id            0
Nice Name             15
Customer Number        0
invoice_id             2
priority_id            2
month                  0
period_from            0
period_to              0
subtotal_JSON          3
Subtotal_Priority      0
Diff_JSON_Priority     3
dtype: int64

In [8]:
null_values_Nice_Name = merged_df[merged_df['Nice Name'].isnull()]
null_values_Nice_Name 

,customer_id,Nice Name,Customer Number,invoice_id,priority_id,month,period_from,period_to,subtotal_JSON,Subtotal_Priority,Diff_JSON_Priority
764,hl_f1239c27,NaN,1878577,EI228033137,lum231371094430,04,01 - 04 - 2022,01 - 05 - 2022,5.45,0.46,4.99
765,hl_f1239c27,NaN,1878577,None,None,05,01 - 05 - 2022,01 - 06 - 2022,5.00,114056.79,114051.79
766,hl_f1239c27,NaN,1878577,EI228053375,lum236667063224,06,01 - 06 - 2022,01 - 07 - 2022,45.01,0.01,45.00
1008,hl_e4f1ed28,NaN,242012,EI228004900,lum223684387936,01,01 - 01 - 2022,01 - 02 - 2022,2000.00,2000.00,0.00
1009,hl_e4f1ed28,NaN,242012,EI228014724,lum226104207620,02,01 - 02 - 2022,01 - 03 - 2022,2000.00,2000.00,0.00
1010,hl_e4f1ed28,NaN,242012,EI228024621,lum228785090485,03,01 - 03 - 2022,01 - 04 - 2022,3545.62,3545.62,0.00
1011,hl_e4f1ed28,NaN,242012,EI228034703,lum231375172181,04,01 - 04 - 2022,01 - 05 - 2022,2000.00,2000.00,0.00
1012,hl_e4f1ed28,NaN,242012,EI228044078,lum234042886860,05,01 - 05 - 2022,01 - 06 - 2022,2000.00,2000.00,0.00
1013,hl_e4f1ed28,NaN,242012,EI228054994,lum236667711023,06,01 - 06 - 2022,01 - 07 - 2022,7482.07,7482.07,0.00
1014,hl_e4f1ed28,NaN,242012,EI228071562,lum239319191759,07,01 - 07 - 2022,01 - 08 - 2022,5449.34,5449.33,0.01


##### Fill the null values in Nice Name with Matched from 'Match 2 - Nice Name'

In [9]:
merged_df = pd.merge(merged_df, xls_customers[['Match 2 - customer_id', 'Match 2 - Nice Name']], 
                     left_on='customer_id', right_on='Match 2 - customer_id', how='left')

merged_df['Nice Name'] = merged_df['Nice Name'].fillna(merged_df['Match 2 - Nice Name'])

merged_df = merged_df.drop(['Match 2 - customer_id', 'Match 2 - Nice Name'], axis=1)


### Checking null values after new matched of NICE NAME


In [10]:
merged_df.isna().sum()

customer_id           0
Nice Name             0
Customer Number       0
invoice_id            2
priority_id           2
month                 0
period_from           0
period_to             0
subtotal_JSON         3
Subtotal_Priority     0
Diff_JSON_Priority    3
dtype: int64

### Check null values of invoice_id

In [12]:
null_values_invoice_id = merged_df[merged_df['invoice_id'].isnull()]
null_values_invoice_id 

,customer_id,Nice Name,Customer Number,invoice_id,priority_id,month,period_from,period_to,subtotal_JSON,Subtotal_Priority,Diff_JSON_Priority
765,hl_f1239c27,Luxury Hype LTD,1878577,None,None,05,01 - 05 - 2022,01 - 06 - 2022,5.0,114056.79,114051.79
803,hl_90440d5b,CROOZ SHOPLIST Inc.,1927878,None,None,03,01 - 03 - 2022,01 - 04 - 2022,209.0,114056.79,113847.79


### Check null values of priority_id

In [13]:
null_values_priority_id = merged_df[merged_df['priority_id'].isnull()]
null_values_priority_id 

,customer_id,Nice Name,Customer Number,invoice_id,priority_id,month,period_from,period_to,subtotal_JSON,Subtotal_Priority,Diff_JSON_Priority
765,hl_f1239c27,Luxury Hype LTD,1878577,None,None,05,01 - 05 - 2022,01 - 06 - 2022,5.0,114056.79,114051.79
803,hl_90440d5b,CROOZ SHOPLIST Inc.,1927878,None,None,03,01 - 03 - 2022,01 - 04 - 2022,209.0,114056.79,113847.79


### Check :
#### it's no priority_id and invoice_id in the source file for this two invoices

### Check null values of Subtotal_Priority

In [14]:
null_values_Subtotal_Priority = merged_df[merged_df['Subtotal_Priority'].isnull()]
null_values_Subtotal_Priority

,customer_id,Nice Name,Customer Number,invoice_id,priority_id,month,period_from,period_to,subtotal_JSON,Subtotal_Priority,Diff_JSON_Priority


In [15]:
null_values_subtotal_JSON = merged_df[merged_df['subtotal_JSON'].isnull()]
null_values_subtotal_JSON

,customer_id,Nice Name,Customer Number,invoice_id,priority_id,month,period_from,period_to,subtotal_JSON,Subtotal_Priority,Diff_JSON_Priority
347,hl_f741fd3a,RedBud Partners Ltd,1297670,EI228009989,lum223713491127,01,01 - 01 - 2022,01 - 02 - 2022,NaN,38000.00,NaN
727,hl_b49b137c,Scale,1871049,EI228072668,lum239336340567,07,01 - 07 - 2022,01 - 08 - 2022,NaN,66199.15,NaN
728,hl_b49b137c,Scale,1871049,EI228083579,lum242121034858,08,01 - 08 - 2022,01 - 09 - 2022,NaN,44423.38,NaN


### Check :
#### it's no subtotal in the source file for this three invoices

In [16]:
merged_df['customer_id'].value_counts()

hl_f68ed861    12
hl_6f64ab28    12
hl_072093e6    12
hl_e588b539    12
hl_d4386d83    12
               ..
hl_c3e1ee30     5
hl_fee7760a     4
hl_44a20cac     4
hl_495466c3     3
hl_f1239c27     3
Name: customer_id, Length: 92, dtype: int64

## check :
### for each one of customer with less than 12  , into each one as single check('Perfect one_Bright_Data' notebook) - Matched to Number of invoices

In [17]:
merged_df.to_csv('Brigh_Data_Merged.csv', index=False) 
